In [1]:
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
import threading
import time
from datetime import datetime
from collections import defaultdict

# Configuration
PAPER_TRADING_PORT = 7497
LIVE_TRADING_PORT = 7496
USE_LIVE_TRADING = False  # Set to True for live trading

class PositionApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.positions = []
        self.positions_received = False
        
    def error(self, reqId, errorCode, errorString, advancedOrderRejectJson="", *args):
        print(f"Error {errorCode}: {errorString}")
        
    def position(self, account, contract, position, avgCost):
        if contract.secType == 'OPT' and position != 0:
            self.positions.append({
                'symbol': contract.symbol,
                'expiry': contract.lastTradeDateOrContractMonth,
                'strike': contract.strike,
                'right': contract.right,
                'position': position,
                'avgCost': avgCost
            })
        
    def positionEnd(self):
        self.positions_received = True

def get_vertical_spreads():
    # Connect to IBKR
    port = LIVE_TRADING_PORT if USE_LIVE_TRADING else PAPER_TRADING_PORT
    app = PositionApp()
    app.connect("127.0.0.1", port, clientId=1)
    
    # Start thread
    thread = threading.Thread(target=app.run)
    thread.daemon = True
    thread.start()
    time.sleep(1)
    
    # Request positions
    app.reqPositions()
    
    # Wait for data
    timeout = 10
    start = time.time()
    while not app.positions_received and (time.time() - start) < timeout:
        time.sleep(0.1)
    
    app.disconnect()
    
    # Process positions into spreads
    spreads = []
    grouped = defaultdict(list)
    
    # Group by symbol, expiry, and option type
    for pos in app.positions:
        key = (pos['symbol'], pos['expiry'], pos['right'])
        grouped[key].append(pos)
    
    # Find vertical spreads
    for (symbol, expiry, right), positions in grouped.items():
        if len(positions) >= 2:
            positions.sort(key=lambda x: x['strike'])
            
            for i in range(len(positions)):
                for j in range(i+1, len(positions)):
                    pos1, pos2 = positions[i], positions[j]
                    
                    # Check for opposite positions (vertical spread)
                    if (pos1['position'] > 0) != (pos2['position'] > 0):
                        spread_type = get_spread_type(pos1, pos2, right)
                        if spread_type:
                            spreads.append(create_spread_row(pos1, pos2, spread_type))
    
    return pd.DataFrame(spreads)

def get_spread_type(pos1, pos2, right):
    """Determine spread type - pos1 is lower strike"""
    if right == 'P':  # Puts
        if pos1['position'] < 0 and pos2['position'] > 0:
            return 'Bull Put'
        elif pos1['position'] > 0 and pos2['position'] < 0:
            return 'Bear Put'
    elif right == 'C':  # Calls
        if pos1['position'] > 0 and pos2['position'] < 0:
            return 'Bull Call'
        elif pos1['position'] < 0 and pos2['position'] > 0:
            return 'Bear Call'
    return None

def create_spread_row(pos1, pos2, spread_type):
    """Create DataFrame row for the spread"""
    symbol = pos1['symbol']
    expiry = format_date(pos1['expiry'])
    low_strike = int(min(pos1['strike'], pos2['strike']))
    high_strike = int(max(pos1['strike'], pos2['strike']))
    
    # Determine which position is which strike
    if pos1['strike'] < pos2['strike']:
        low_pos, high_pos = pos1, pos2
    else:
        low_pos, high_pos = pos2, pos1
    
    title = f"{symbol} {expiry} {low_strike}/{high_strike} {spread_type}"
    low_leg = f"{symbol} {expiry} {low_strike} {pos1['right']}"
    high_leg = f"{symbol} {expiry} {high_strike} {pos1['right']}"
    
    # Calculate market value (simplified)
    market_value = (float(low_pos['avgCost']) * float(low_pos['position']) + 
                   float(high_pos['avgCost']) * float(high_pos['position'])) * 100
    
    return {
        'Position Title': title,
        'Low Strike Leg': low_leg,
        'Low Strike Position': int(low_pos['position']),
        'High Strike Leg': high_leg,
        'High Strike Position': int(high_pos['position']),
        'Market Value': round(market_value, 2),
        'Spread Type': spread_type,
        'Expiry Date': datetime.strptime(pos1['expiry'], '%Y%m%d').date()
    }

def format_date(expiry_str):
    """Convert YYYYMMDD to 'Mon DD'YY' format"""
    dt = datetime.strptime(expiry_str, '%Y%m%d')
    return dt.strftime("%b %d'%y")

if __name__ == "__main__":
    print(f"Connecting to IBKR on port {LIVE_TRADING_PORT if USE_LIVE_TRADING else PAPER_TRADING_PORT}")
    
    df = get_vertical_spreads()
    
    if not df.empty:
        print(f"\nFound {len(df)} vertical spreads:")
        print(df.to_string(index=False))
    else:
        print("No vertical spreads found")

/tmp/ipykernel_10390/3873110129.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Connecting to IBKR on port 7497
Error 0: 2104
Error 0: 2104
Error 0: 2104
Error 0: 2106
Error 0: 2106
Error 0: 2106
Error 0: 2158

Found 3 vertical spreads:
                  Position Title        Low Strike Leg  Low Strike Position       High Strike Leg  High Strike Position  Market Value Spread Type Expiry Date
GOOGL Jun 27'25 145/155 Bear Put GOOGL Jun 27'25 145 P                    1 GOOGL Jun 27'25 155 P                    -1      -6038.13    Bear Put  2025-06-27
    WMT Jun 27'25 84/94 Bear Put    WMT Jun 27'25 84 P                    1    WMT Jun 27'25 94 P                    -1       -966.13    Bear Put  2025-06-27
 TSLA Jun 27'25 275/285 Bear Put  TSLA Jun 27'25 275 P                    1  TSLA Jun 27'25 285 P                    -1     -22372.13    Bear Put  2025-06-27


In [2]:
spreads

NameError: name 'spreads' is not defined